# Segmented Representations

One common representation in evolutionary algorithms (EA) is that of a "segmented representation."  That is, each individual is comprised of a sequence of segments, which are themselves fixed-length sequences, and are usually binary, but needn't be.  Each segment represents a salient feature, such as a rule in a Pitt Approach system, or a convolutional layer and its hyperparameters, as is the case for Multi-node Evolutionary Neural Networks for Deep Learning (MENNDL).

There are two broad categories for these systems: those that have a fixed number of such segments, as is the case currently for MENNDL, and a dynamic number of segments, as is the case for Pitt Approach classifiers.

In this notebook we look at LEAP support for segmented representations, starting with initializers and decoders, and then looking at the mutation pipeline operator.  We then plug all that into a simple EA example.


In [1]:
import sys
import random
import functools
from pprint import pprint, pformat
import numpy as np
from toolz import pipe

from leap_ec.individual import Individual
from leap_ec.ops import pool, cyclic_selection, clone

from leap_ec.segmented_rep.initializers import create_segmented_sequence
from leap_ec.segmented_rep.decoders import SegmentedDecoder
from leap_ec.segmented_rep.ops import apply_mutation, add_segment, remove_segment, copy_segment

from leap_ec.binary_rep.initializers import create_binary_sequence
from leap_ec.binary_rep.ops import genome_mutate_bitflip
from leap_ec.binary_rep.decoders import BinaryToIntDecoder

from leap_ec.real_rep.initializers import create_real_vector
from leap_ec.real_rep.ops import genome_mutate_gaussian

## Binary genomes

We first look at segmented representations with segments that use a binary representaion.

In [2]:
# Create a genome of four segments of five binary digits.
seg = create_segmented_sequence(4, create_binary_sequence(5))
print(seg)

[[1, 1, 0, 1, 1], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1], [1, 0, 0, 1, 0]]


In [3]:
# Now create five genomes of varying length by passing in a function for `length` that provides an
# integer drawn from a distribution.
seqs = [] # Save sequences for next step
for i in range(5):
    seq = create_segmented_sequence(functools.partial(random.randint, a=1,b=5), create_binary_sequence(5))
    print(i, seq)
    seqs.append(seq)

0 [[1, 1, 0, 0, 1], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 0]]
1 [[1, 0, 1, 1, 0], [0, 1, 1, 1, 1]]
2 [[1, 1, 0, 0, 0]]
3 [[0, 0, 1, 1, 0], [0, 0, 0, 1, 1]]
4 [[0, 1, 1, 0, 0], [1, 0, 1, 1, 1], [0, 0, 0, 0, 0]]


Now let's see about decoding those segments.  The segmented representation relies on a secondary decoder that's applied to each segment.  In this case, we'll just use a simple binary to int decoder on the segments we created in the previous step.

In [4]:
# We want each segment to have two integers from the five bits.
decoder = SegmentedDecoder(BinaryToIntDecoder(2,3)) 

for i, seq in enumerate(seqs):
    vals = decoder.decode(seq)
    print(i, vals)

0 [[3, 1], [0, 4], [0, 2], [1, 6]]
1 [[2, 6], [1, 7]]
2 [[3, 0]]
3 [[0, 6], [0, 3]]
4 [[1, 4], [2, 7], [0, 0]]


In [5]:
# And now for mutation, which shows that, on average, a single value is changed in an example individual.  The
# takeaway here is that segmented mutation just uses a mutator from another representation and naively applies it.

original = Individual([[0,0],[1,1]])
print('original:', original)
mutated = next(apply_mutation(iter([original]),mutator=genome_mutate_bitflip))
print('mutated:', mutated)

original: [[0, 0], [1, 1]]
mutated: [[1, 0], [0, 0]]


## Real-valued genomes

Now we demonstrate the same process using a real-valued representation.

In [6]:
# Create five segmented sequences that vary from 1 to 3 segments
bounds = ((-5.12,5.12), (-1,1), (-10,10)) # three reals and their respective bounds for sampling
seqs = []
for i in range(5):
    seq = create_segmented_sequence(functools.partial(random.randint, a=1,b=3), 
                                    create_real_vector(bounds))
    seqs.append(seq)

# Just for fun, now add a genome that has exactly 5 segments
seqs.append(create_segmented_sequence(5, create_real_vector(bounds)))

for i, s in enumerate(seqs):
    print(i, pformat(s, indent=2))

0 [ [-1.8973586637375401, 0.5429358612872954, 0.0052846033063165265],
  [-0.5754921007925509, 0.5104225758006147, 6.4648271768005365],
  [-3.4834242225033405, -0.26507258048532023, 1.935257535128457]]
1 [ [4.98067083675486, 0.4223739312771899, -5.925152673270164],
  [-2.577998425670687, -0.35010930955341757, -2.617175140240022]]
2 [[-0.5684113423076393, -0.04963893132542441, 5.449940202099189]]
3 [ [-0.316042198876505, -0.13466716975734516, 9.3410379221799],
  [0.6957597935205024, -0.6893605496835147, -1.9398972704320094],
  [-0.34054363261319764, -0.38336623906202805, -5.64731803306884]]
4 [ [0.11732469015839264, -0.7359217090528429, -3.099837594705037],
  [-4.8520935601328015, -0.5778270903000526, 3.0948850786428643]]
5 [ [-2.556130348154271, -0.7040061334657957, 1.6982386488629704],
  [1.491859774130151, -0.047509351710018244, 9.321873468073917],
  [-3.998912637647323, -0.6493184695058707, -8.604538374025111],
  [-1.6518550600335664, 0.5130972581544762, -2.4723515656917634],
  [1.63

Now we repeat the application of the segmented mutation operator, but this time to real-valued genomes.

In [27]:
original = Individual([[0.0,0.0],[1.0,1.0],[-1.0,0.0]])
print('original:', original)
mutated = next(apply_mutation(iter([original]), expected_num_mutation=3, mutator=genome_mutate_gaussian(std=1.0)))
print('mutated:', mutated)

original: [[0.0, 0.0], [1.0, 1.0], [-1.0, 0.0]]
mutated: [[2.076157969470278, 0.14933710513356127], [0.23274025747200955, 1.0], [-1.7865022095622027, 1.720654038451787]]


# Other pipeline operators

Besides the aformentioned `apply_mutation`, segmented representations have other pipeline operators, which are:

* `add_segment()`, possibly add a new segment
* `remove_segment()`, possibly remove a segment
* `copy_segment()`, possibly select and copy an existing segment


In [8]:
# demonstrate pipe by running existing sequence through a number of operators
pop = [Individual([[0,0],[1,1]]) for x in range(5)]
print('pop:', pformat(pop))
new_pop = pipe(pop, 
               cyclic_selection,
               clone,
               remove_segment(probability=1.0), 
               pool(size=len(pop)))
print('new_pop:', pformat(new_pop))

pop: [Individual([[0, 0], [1, 1]], None, None),
 Individual([[0, 0], [1, 1]], None, None),
 Individual([[0, 0], [1, 1]], None, None),
 Individual([[0, 0], [1, 1]], None, None),
 Individual([[0, 0], [1, 1]], None, None)]
new_pop: [Individual([[0, 0]], None, None),
 Individual([[0, 0]], None, None),
 Individual([[0, 0]], None, None),
 Individual([[1, 1]], None, None),
 Individual([[0, 0]], None, None)]


In [9]:
# demonstrate pipe by running existing sequence through a number of operators
pop = [Individual([[0,0],[1,1]]) for x in range(5)]
print('pop:', pformat(pop, indent=5))
new_pop = pipe(pop, 
               cyclic_selection,
               clone,
               copy_segment(probability=1.0),
               pool(size=len(pop)))
print('new_pop:', pformat(new_pop, indent=9))

pop: [    Individual([[0, 0], [1, 1]], None, None),
     Individual([[0, 0], [1, 1]], None, None),
     Individual([[0, 0], [1, 1]], None, None),
     Individual([[0, 0], [1, 1]], None, None),
     Individual([[0, 0], [1, 1]], None, None)]
new_pop: [        Individual([[0, 0], [0, 0], [1, 1]], None, None),
         Individual([[1, 1], [0, 0], [1, 1]], None, None),
         Individual([[0, 0], [1, 1], [1, 1]], None, None),
         Individual([[1, 1], [0, 0], [1, 1]], None, None),
         Individual([[0, 0], [1, 1], [1, 1]], None, None)]


In [10]:
# lastly, demonstrate add_segment, which generates an entirely new segment
test_sequence = [12345]  # just an arbitrary sequence for testing

def gen_sequence():
    """ return an arbitrary static test_sequence """
    return test_sequence

pop = [Individual([[0,0],[1,1]]) for x in range(5)]
print('pop:', pformat(pop, indent=5))

new_pop = pipe(pop, 
               cyclic_selection,
               clone,
               add_segment(seq_initializer=gen_sequence, probability=1.0),
               pool(size=len(pop)))
print('new_pop:', pformat(new_pop, indent=9))

pop: [    Individual([[0, 0], [1, 1]], None, None),
     Individual([[0, 0], [1, 1]], None, None),
     Individual([[0, 0], [1, 1]], None, None),
     Individual([[0, 0], [1, 1]], None, None),
     Individual([[0, 0], [1, 1]], None, None)]
new_pop: [        Individual([[0, 0], [12345], [1, 1]], None, None),
         Individual([[0, 0], [1, 1], [12345]], None, None),
         Individual([[12345], [0, 0], [1, 1]], None, None),
         Individual([[12345], [0, 0], [1, 1]], None, None),
         Individual([[0, 0], [1, 1], [12345]], None, None)]
